### Zadanie 1.
Napisz w dowolnym języku zwyczajną (wolną) dyskretną transformatę Fouriera.

In [1]:
import numpy as np
def DFT_slow(x):
    x = np.asarray(x, dtype=float)
    N = x.shape[0]
    
    n = np.arange(N)
    k = n.reshape((N, 1))
    M = np.exp(-2j * np.pi * k * n / N)
    return np.dot(M, x)

In [2]:
x = np.random.random(1024)
np.allclose(DFT_slow(x), np.fft.fft(x))

True

### Zadanie 2.
Wykorzystaj implementację z zadania 1.
do napisania szybkiej wersji transformaty (używając pomysłu z algorytmu Cooleya-Tukeya).

In [3]:
def FFT(x):
    x = np.asarray(x, dtype=float)
    N = x.shape[0]
    
    if N % 2 > 0:
        raise ValueError("size of x must be a power of 2, but is ", N)
    elif N <= 16:
        return DFT_slow(x)
    else:
        X_even = FFT(x[::2])
        X_odd = FFT(x[1::2])
        factor = np.exp(-2j * np.pi * np.arange(N) / N)
        return np.concatenate([X_even + factor[:N // 2] * X_odd, X_even + factor[N // 2:] * X_odd])

In [4]:
x = np.random.random(1024)
np.allclose(FFT(x), np.fft.fft(x))

True

In [5]:
%timeit DFT_slow(x)
%timeit FFT(x)
%timeit np.fft.fft(x)

116 ms ± 5.58 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
3.82 ms ± 238 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
29.9 µs ± 3.82 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


### Zadanie 3.
Przetestuj implementację z zadania 2. do wykonania analizy szeregu czasowego:
1. Znajdź dane przedstawiające jakiś szereg czasowy.
2. Załaduj je do programu (polecany: Python + Pandas, ale dowolna metoda jest okej, w tym języki R oraz Julia).
3. Zobacz, czy wykonanie analizy Fouriera na tych danych ma sens -- być może trzeba pogrupować je w równe odstępy (zob: funkcja resample w pakiecie Pandas).
4. Narysuj wykres częstotliwości i postaraj się opisać jaka jest główna składowa.

In [6]:
import plotly.plotly as py
import plotly.graph_objs as go

t = np.linspace(0, 0.5, 512)
s = np.sin(40 * 2 * np.pi * t) + 0.5 * np.sin(90 * 2 * np.pi * t) + 0.3 * np.cos(180 * 2 * np.pi * t)
T = t[1] - t[0]
N = s.size
f = np.linspace(0, 1 / T, N)

fft = FFT(s)

data = [go.Scatter( x=f[:N // 2], y=np.abs(fft)[:N // 2] * 1 / N )]
py.iplot(data, filename='xdd')

In [7]:
import plotly.plotly as py
import plotly.graph_objs as go
import pandas as pd

url = "global_temperature_monthly.csv"
df = pd.read_csv(url, nrows=1024)
s = df['Mean']

fft = FFT(s)
data = [go.Scatter( x=df['Date'], y=np.abs(fft)[1:N // 2])]
py.iplot(data, filename='fft')

### Pytanie otwarte
Czy transformata Fouriera może zostać wykorzystana do przewidywania szeregów czasowych:?